<a href="https://colab.research.google.com/github/HelenSaraAlex/Cosmic_Image_classifier-using-CNN/blob/main/CosmicObjectIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16

In [ ]:
plt.rc('figure', autolayout = True)
plt.rc('axes', labelweight = 'bold', titleweight = 'bold', titlesize = 18, titlepad = 10, labelsize = 'large')
plt.rc('image', cmap = 'magma')
warnings.filterwarnings("ignore")

In [ ]:
def seed_set(seed = 31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC-OPS'] = '1'
seed_set(31415)

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dir = '/content/drive/MyDrive/Identify_planet/train'
valid_dir = '/content/drive/MyDrive/Identify_planet/valid'

print("Training directory:",train_dir)
print("Validation directory:",valid_dir)

ds_train_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True,
)

ds_valid_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False,
)


Training directory: /content/drive/MyDrive/Identify_planet/train
Validation directory: /content/drive/MyDrive/Identify_planet/valid
Found 368 files belonging to 8 classes.
Found 368 files belonging to 8 classes.


In [ ]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

In [ ]:
pretrained_base = VGG16(weights = 'imagenet',include_top = False, input_shape = (128,128,3))

pretrained_base.trainable = False

In [ ]:
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation = 'softmax')
])
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)


history = model.fit(
    ds_train,validation_data = ds_valid,
    epochs = 70,
    verbose = 1
)

Epoch 1/70
6/6 [==============================] - 2s 220ms/step - loss: 1.7944 - accuracy: 0.4484 - val_loss: 0.3734 - val_accuracy: 0.8913
Epoch 2/70
6/6 [==============================] - 1s 197ms/step - loss: 0.4296 - accuracy: 0.8288 - val_loss: 0.1182 - val_accuracy: 1.0000
Epoch 3/70
6/6 [==============================] - 1s 157ms/step - loss: 0.1968 - accuracy: 0.9212 - val_loss: 0.0658 - val_accuracy: 1.0000
Epoch 4/70
6/6 [==============================] - 1s 156ms/step - loss: 0.1405 - accuracy: 0.9484 - val_loss: 0.0308 - val_accuracy: 1.0000
Epoch 5/70
6/6 [==============================] - 1s 158ms/step - loss: 0.0739 - accuracy: 0.9864 - val_loss: 0.0207 - val_accuracy: 1.0000
Epoch 6/70
6/6 [==============================] - 1s 155ms/step - loss: 0.0461 - accuracy: 0.9946 - val_loss: 0.0158 - val_accuracy: 1.0000
Epoch 7/70
6/6 [==============================] - 1s 156ms/step - loss: 0.0339 - accuracy: 0.9973 - val_loss: 0.0101 - val_accuracy: 1.0000
Epoch 8/70
6/6 [====

In [ ]:
labels = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [ ]:
def preprocess_img(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [128,128])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [ ]:
def predict_class(image_path):
    image = preprocess_img(image_path)
    image = tf.expand_dims(image, axis = 0)
    predictions = model.predict(image)

    predicted_index = tf.argmax(predictions[0]).numpy()
    predicted_class = labels[predicted_index]
    return predicted_class

In [ ]:
#test
image_path='/content/drive/MyDrive/Identify_planet/test/Saturn/Saturn1755_jpg.rf.d67c8089c411267c2db60ddb307b2869.jpg'
planet=predict_class(image_path)
print("predicted planet:",planet)

1/1 [==============================] - 0s 18ms/step
predicted planet: Saturn
